This re-iteration is based on department 37's Use-of-force data.

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Step 3 Data Preparation').getOrCreate()
from pyspark.sql.functions import *
from functools import reduce
from pyspark.ml.feature import StringIndexer

# 3.1

In [2]:
df3 = spark.read.csv('37-00049_UOF-P_2016_prepped.csv', header=True, inferSchema=True)

In [3]:
old_col = df3.schema.names #original column headings
new_col = [i for i in df3.head(1)[0]] #new column headings
df3 = reduce(lambda df3, idx: df3.withColumnRenamed(old_col[idx], new_col[idx]), range(len(old_col)), df3) #replace with lambda function

In [4]:
df3 = df3.where("OCCURRED_D != 'OCCURRED_D'")

In [5]:
df3 = df3[["OffSex", "OffRace", "INCIDENT_DATE_LESS_", "OFF_INJURE_DESC", "OFF_HOSPIT", "CitRace", "CitSex", 
           "SUBJ_INJURE_DESC", "CIT_ARREST", "CIT_INFL_A", "CITChargeT", "SERVICE_TY", "UOF_REASON", "ForceType1", "ForceType2", 
           "ForceType3", "ForceType4","ForceType5", "ForceType6", "ForceType7", "ForceType8", "ForceType9", "ForceType10"]]

In [6]:
df3.show(5)

+------+-------+-------------------+--------------------+----------+--------+------+--------------------+----------+-------------------+--------------------+----------+----------+--------------------+---------------+----------+----------+----------+----------+----------+----------+----------+-----------+
|OffSex|OffRace|INCIDENT_DATE_LESS_|     OFF_INJURE_DESC|OFF_HOSPIT| CitRace|CitSex|    SUBJ_INJURE_DESC|CIT_ARREST|         CIT_INFL_A|          CITChargeT|SERVICE_TY|UOF_REASON|          ForceType1|     ForceType2|ForceType3|ForceType4|ForceType5|ForceType6|ForceType7|ForceType8|ForceType9|ForceType10|
+------+-------+-------------------+--------------------+----------+--------+------+--------------------+----------+-------------------+--------------------+----------+----------+--------------------+---------------+----------+----------+----------+----------+----------+----------+----------+-----------+
|  Male|  Black|                  2|No injuries noted...|        No|   Black|Femal

# 3.2

In [6]:
missing_values = ["UNKNOWN", "NO DATA ENTERED", -1, "No Data", "Unknown", "unknown"]
for i in missing_values:
    for j in df3.columns: #go through all columns to replace codes with null
        df3 = df3.withColumn(j, when(df3[j] == i, None).otherwise(df3[j]))

In [7]:
x = []
for c in df3.columns:
    x.append((count(when(df3[c].isNull(), c))*100.0/df3.count()).alias(c))
col_emp = df3.select(x).collect()
emp_col = [row.asDict() for row in col_emp] #store each column as key and its emptiness as value
emp_col = emp_col[0]
col_null_g_25p=[i for i in emp_col if emp_col[i] > 25]
df3 = df3.drop(*col_null_g_25p)
df3.show(5)

+------+-------+-------------------+--------------------+----------+--------+------+--------------------+----------+-------------------+--------------------+----------+----------+--------------------+
|OffSex|OffRace|INCIDENT_DATE_LESS_|     OFF_INJURE_DESC|OFF_HOSPIT| CitRace|CitSex|    SUBJ_INJURE_DESC|CIT_ARREST|         CIT_INFL_A|          CITChargeT|SERVICE_TY|UOF_REASON|          ForceType1|
+------+-------+-------------------+--------------------+----------+--------+------+--------------------+----------+-------------------+--------------------+----------+----------+--------------------+
|  Male|  Black|                  2|No injuries noted...|        No|   Black|Female|Non-Visible Injur...|       Yes|  Mentally unstable|               APOWW|    Arrest|    Arrest|Hand/Arm/Elbow St...|
|  Male|  White|                 17|       Sprain/Strain|       Yes|Hispanic|  Male|No injuries noted...|       Yes|  Mentally unstable|               APOWW|    Arrest|    Arrest|         Joint Lo

Only null values for subject race and type of service are considered for rejection.

In [9]:
df3 = df3.na.drop(subset=["CitRace", "SERVICE_TY"])

Null values in other fields such as Reason for force, etc. are replaced with "No reason".

In [11]:
df3 = df3.na.fill('No reason', subset=['CIT_INFL_A', 'UOF_REASON'])

In [ ]:
df3.toPandas().to_csv("Department 37 prepped.csv")

In [12]:
df3.show(5)

+------+-------+-------------------+--------------------+----------+--------+------+--------------------+----------+-------------------+--------------------+----------+----------+--------------------+
|OffSex|OffRace|INCIDENT_DATE_LESS_|     OFF_INJURE_DESC|OFF_HOSPIT| CitRace|CitSex|    SUBJ_INJURE_DESC|CIT_ARREST|         CIT_INFL_A|          CITChargeT|SERVICE_TY|UOF_REASON|          ForceType1|
+------+-------+-------------------+--------------------+----------+--------+------+--------------------+----------+-------------------+--------------------+----------+----------+--------------------+
|  Male|  Black|                  2|No injuries noted...|        No|   Black|Female|Non-Visible Injur...|       Yes|  Mentally unstable|               APOWW|    Arrest|    Arrest|Hand/Arm/Elbow St...|
|  Male|  White|                 17|       Sprain/Strain|       Yes|Hispanic|  Male|No injuries noted...|       Yes|  Mentally unstable|               APOWW|    Arrest|    Arrest|         Joint Lo